In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from collections import Counter
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score,KFold
import numpy as np
%matplotlib inline

In [27]:

class Model():
    def __init__(self,data,zscore=False,out=False,smote = False,adasyn=False,test=False):
        self.df = data
        self.data = data
        self.word_num()
        if zscore==True:
            self.df = data.iloc[:,1:-2].apply(stats.zscore)
            if test == False:
                self.df['quality'] = data['quality']
            self.df['id'] = data['id']
            self.df['type'] = data['type']
        if zscore==True & out ==True:
            for i in self.df.iloc[:,:-3].columns:
                self.df = self.df[(self.df[i] > -2) & (self.df[i] < 2)]
        if test ==False:
            self.X = data.drop(['id', 'quality'],axis=1)
            self.y = data['quality']
        if smote==True:
            self.X, self.y = SMOTE(random_state=0).fit_resample(self.X,self.y)
        elif adasyn==True:
            self.X, self.y = ADASYN(random_state=0).fit_resample(self.X,self.y)
        self.X_train,self.X_test,self.y_train,self.y_test = train_test_split(self.X,self.y,
                                                                            test_size=0.4,
                                                                            random_state=42)
    def Logistic(self,linear = 'liblinear'):
        model = LogisticRegression(solver = linear)
        model.fit(self.X_train,self.y_train)
        prediction = model.predict(self.X_test)
        score = self.evaluate_model(self.X_test,self.y_test,model)
        print(f"모델의 정확도는 {np.mean(score)}% 입니다")
        self.__init__(self.data)
        
        return model
    def grid_model(self,model_type,param=None):
        #model = GridSearchCV(model_type,param,scoring = 'accuracy',cv=3)
        model = model_type
        model.fit(self.X_train,self.y_train)
        prediction = model.predict(self.X_test)
        score = self.evaluate_model(self.X_test,self.y_test,model)
        self.__init__(self.data)
        print(f"모델의 정확도는 {np.mean(score)}% 입니다")
        return model
    def count_plot(self): 
        counter = Counter(self.y)
        for k,v in counter.items():
            print('Class=%d, n=%d (%.3f%%)' % (k, v, v / len(y) * 100))
        pyplot.bar(counter.keys(), counter.values())
        pyplot.show()
    def word_num(self):
        word_to_num = {'white' : 0, 'red' : 1}
        self.df['type'] = self.df['type'].replace(word_to_num)
    def evaluate_model(self,X, y, model):
        cv = KFold(n_splits=10, shuffle=True)
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        return scores

In [29]:
train_path = '../train/train.csv'
test_path = '../test/test.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)


In [20]:
train_model = Model(train_df,zscore=True,out=True,).Logistic()



모델의 정확도는 0.5248800295311924% 입니다


In [31]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()

param_grid ={'max_depth':(1,2,3,4,5,6,7,8,9,10)}

model = Model(train_df,zscore=True,out=True,smote=True)
train_model = model.grid_model(regressor,param_grid)

모델의 정확도는 0.6096750111979297% 입니다


In [30]:
import xgboost as xgb
xgbr = xgb.XGBRegressor()
model = Model(train_df,zscore=True,out=True,smote=True)
train_model = model.grid_model(xgbr,param_grid)

모델의 정확도는 nan% 입니다


In [21]:

test = pd.read_csv('../test/test.csv')
word_to_num = {"white":0, "red":1}

test['type'] = test['type'].replace(word_to_num)
test =  test.apply(stats.zscore)
prediction = train_model.predict(test.drop('id', axis=1))


In [182]:
test =  test.apply(stats.zscore)

In [190]:
prediction = train_model.predict(test.drop('id', axis=1))


In [191]:
submission = pd.read_csv('../sample_submission.csv')
submission['quality'] = prediction
submission.to_csv("../submission/logistic.csv", index=False)